In [2]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'aajkikhabar_com'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_rss_page = 0
publish_source = 'aajkikhabar.com'
country = 'India'
language = 'Hindi'
images_path = []


rss_pages = c.download_page('https://www.aajkikhabar.com/feed/')

for _ in rss_pages.split('href=')[1:3]:
    
    rss_url = c.scrap('"(.*?)"', _)
    url = c.download_page(rss_url)
    
    if url.startswith('Unable to fetch'):
        logger.info(url) # writes error message with error code
        unable_to_fetch_rss_page += 1
        continue    

    for i in url.split('<item>')[1:]:
        
         # source_link
        source_link = c.scrap('<link>(.*?)</link>', i)
        print(source_link)
        
        
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue    
            
        time.sleep(random.randint(1,3))
        
        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_url += 1
                continue  
        
        # source_headline
        source_headline = c.scrap('<title>(.*?)</title>', page)
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue

        logger.info(f'Fetching {source_link}\n')
        
         # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('<pubDate>\w+,\s+(.*?)\+', i)
            date_time_obj = datetime.strptime(date_time_str, '%d %b %Y %H:%M:%S')
            # There is no time difference between India and Sri Lanka
            ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue
            
                        
         # journalist   
        journalist = c.scrap('rel="author">(.*?)</a>', page)
        print(journalist)
        if not journalist: journalist = 'NA'
                            
         # source_content          
        source_content = c.scrap('id="mvp-content-body-top".*?<p>(.*?)</div>', page)
        source_content = c.strip_html(source_content)
                        
        # current date and time        
        harvest_time = datetime.now().strftime("%H:%M:%S")
        
          # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1

logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_page}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')



        


https://www.aajkikhabar.com/feed/
https://www.aajkikhabar.com/cm-yogi-tweet-on-holi-festival/
सीएम योगी ने प्रदेशवासियों को दी होली शुभकामनाएं, कही ये बात - Aaj Ki Khabar
Mohammad Faique
लखनऊ। देशभर में आज रंगों का त्योहार होली धूमधाम से मनाया जा रहा है। उत्तर प्रदेश में भी इस पावन पर्व को सभी आम और खास लोग उत्साह से मना रहे हैं।
यूपी के मुख्यमंत्री योगी आदित्यनाथ ने होली के अवसर पर प्रदेशवासियों को शुभकामनाएं दी हैं। साथ ही बढ़ते संक्रमण को देखते हुए गाइडलाइंस का पालन करने की अपील भी की है।
मुख्यमंत्री ने कहा- ‘प्रदेशवासियों को होली की शुभकामनाएं देता हूं। उत्साह और उमंग का यह पर्व प्रत्येक प्रदेशवासी के जीवन में सुख, समृद्धि और मंगल का कारक बने। मेरी प्रदेशवासियों से अपील है कि हम हर्षोल्लास के इस पर्व को कोरोना संक्रमण का कारण नहीं बनने दें।’ सीएम योगी ने अपील की कि प्रदेश में कोरोना की दूसरी लहर देखने को मिल रही है ऐसे में सभी लोगों सावधानी बरतनी चाहिए।
बता दें कि कोरोना के बढ़ते ग्राफ को देखते हुए सरकार की तरफ से भी त्यौहारों को मनाने को लेकर कुछ दिशानिर्देश जारी किए गए हैं। गाइडलाइ